## Turing Machine Simulator

<pre>
CSC 427, Semester 232
Burton Rosenberg
University of Miami
5 march 2023

Copyright 2023 (c) Burton Rosenberg. All rights reserved.
</pre>




### A Simple RE recognizer.

An example for the syntax is the following program recognizing a+b+c+. 

Note the call to compute_tm requires a string, a parameter giving maximum number of steps
permitted to the computation, and an optional verbose parameter.

In [1]:
tm_abc= """# a TM program to recognize a+b+c+

accept: A
reject: R
start: q0

state: q0    
    a a r q0  # should start with an a, and loop over further a's
    b b r q1  # until a b.
              # note: missing transtions cause the machine to reject (c or _)
    
state: q1
    b b r q1  # continue over further b's
    c c r q2  # until a c.
    
state: q2
    c c r q2  # continue over further c's
    _ _ r A   # until the end of the tape

"""

# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

from turing_machine_sim import *

tm = MachineParser.create_from_description(tm_abc)
tm.compute_tm('aabbcc', max_steps, verbose=verbose)
tm.compute_tm('abca', max_steps, verbose=verbose)



0 [q0]	[a]abbcc_
1 [q0]	a[a]bbcc_
2 [q0]	aa[b]bcc_
3 [q1]	aab[b]cc_
4 [q1]	aabb[c]c_
5 [q2]	aabbc[c]_
6 [q2]	aabbcc[_]
7 [A]	aabbcc_[_]
accept (ok)
0 [q0]	[a]bca_
1 [q0]	a[b]ca_
2 [q1]	ab[c]a_
3 [q2]	abc[a]_
reject (transition missing)


False

## The copy language

Accept strings of the form `w&w` where `w` is a string over `{a,b}^*`.

This uses the zig-zag marking approach.

In [2]:
tm_erase = """# w&w

accept: A
reject: R
start: s

state: s   # write a tape endmarker
    a x r looka1 # x it off and look for a matching a
    b x r lookb1 # x it off and look for a matching b
    & : r end1   # left w is all x'ed off
    
state: looka1
    : : r looka1 # move right until &
    & : r looka2
    _ : r R      # no &
    
state: looka2
    : : r looka2 # move right until a matching a
    a x l back1  # found, go back for another character
    _ : r R      # no a
    
state: lookb1
    : : r lookb1 # move right until &
    & : r lookb2
    _ : r R      # no &
    
state: lookb2
    : : r lookb2 # move right until a matching b
    b x l back1  # found, go back for another character
    _ : r R      # no b

state: back1
    : : l back1 # find the &
    & : l back2 
    
state: back2
    : : l back2 # find the x
    x x r s 

state: end1
    x x r end1
    _ _ r A

"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_erase)
tm.compute_tm('aab&aab', max_steps, verbose=verbose)
tm.compute_tm('aab&abb', max_steps, verbose=verbose)



0 [s]	[a]ab&aab_
1 [looka1]	x[a]b&aab_
2 [looka1]	xa[b]&aab_
3 [looka1]	xab[&]aab_
4 [looka2]	xab&[a]ab_
5 [back1]	xab[&]xab_
6 [back2]	xa[b]&xab_
7 [back2]	x[a]b&xab_
8 [back2]	[x]ab&xab_
9 [s]	x[a]b&xab_
10 [looka1]	xx[b]&xab_
11 [looka1]	xxb[&]xab_
12 [looka2]	xxb&[x]ab_
13 [looka2]	xxb&x[a]b_
14 [back1]	xxb&[x]xb_
15 [back1]	xxb[&]xxb_
16 [back2]	xx[b]&xxb_
17 [back2]	x[x]b&xxb_
18 [s]	xx[b]&xxb_
19 [lookb1]	xxx[&]xxb_
20 [lookb2]	xxx&[x]xb_
21 [lookb2]	xxx&x[x]b_
22 [lookb2]	xxx&xx[b]_
23 [back1]	xxx&x[x]x_
24 [back1]	xxx&[x]xx_
25 [back1]	xxx[&]xxx_
26 [back2]	xx[x]&xxx_
27 [s]	xxx[&]xxx_
28 [end1]	xxx&[x]xx_
29 [end1]	xxx&x[x]x_
30 [end1]	xxx&xx[x]_
31 [end1]	xxx&xxx[_]
32 [A]	xxx&xxx_[_]
accept (ok)
0 [s]	[a]ab&abb_
1 [looka1]	x[a]b&abb_
2 [looka1]	xa[b]&abb_
3 [looka1]	xab[&]abb_
4 [looka2]	xab&[a]bb_
5 [back1]	xab[&]xbb_
6 [back2]	xa[b]&xbb_
7 [back2]	x[a]b&xbb_
8 [back2]	[x]ab&xbb_
9 [s]	x[a]b&xbb_
10 [looka1]	xx[b]&xbb_
11 [looka1]	xxb[&]xbb_
12 [looka2]	xxb&[x]bb_
13 [look

False

### Erasing the Tape

Accept everything, while erasing the tape leaving the head at the start of the tape.

A Turing Machine opens up another aspect of computation. So far, the algorithms in this
class have only a boolean result: true or false for membership of an element in a set.
(In fact, membership in a subset of a set, as we always assume the input is of proper form.)

This example intrduces another meaning for computation. It is the more common meaning of
"doing something" &mdash; in this case, the TM erases the tape.

In [3]:
tm_erase = """# erase the tape

accept: A
reject: R
start: q0

state: q0    # write a tape endmarker
    : $ R q1
    
state: q1
    : : r q1 # more right not replacing symbols
    _ _ l q2 # begin the erase while returning
    
state: q2
    : _ l q2 # erase symbols
    $ _ N A  # stop at the endmarker
"""


# verbose = 'none'
verbose = 'explain'
# verbose = 'verbose'

max_steps = 100

tm = MachineParser.create_from_description(tm_erase)
tm.compute_tm('abc', max_steps, verbose=verbose)


0 [q0]	[a]bc_
1 [q1]	$[b]c_
2 [q1]	$b[c]_
3 [q1]	$bc[_]
4 [q2]	$b[c]_
5 [q2]	$[b]__
6 [q2]	[$]___
7 [A]	[_]___
accept (ok)


True